## Comparing models for our different tasks

In this Notebook, we are going to use another model, Flan-T5-large (This time hosted locally in the Red Hat OpenShift AI cluster) in parallel to GPT-4 and see how it behaves.

Flan-T5-Large is indeed smaller, will run without a GPU and use only 4 GB of RAM, but is it up to the task?

### Library imports

First we will import the libraries we need, they are already installed on our workbench image so no need to run `pip install`.

In [ ]:
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_openai import AzureChatOpenAI
from langchain_community.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

if load_dotenv():
    print("Found Azure OpenAI Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else:
    print("No file .env found")

### Langchain pipeline

We are now going to define two different LLM endpoints, and two different Langchain pipelines.

In [ ]:
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")

optional_params = {
  "top_p": 0.92,
  "presence_penalty":1.03
}

llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_DEPLOYMENT"),
    max_tokens=512,
    temperature=0.01,
    model_kwargs=optional_params,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Flan-T5-Small LLM Inference Server URL
inference_server_url_flan_t5 = "http://llm-flant5.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm_flant5 = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flan_t5,
    max_new_tokens=96,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

The **template** will be the same for both models.

In [ ]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text, then ask a question about it. Give a precise and as concise as possible answer to this question.

### TEXT:
{text}

### QUESTION:
{query}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

And we can now create two **conversation** objects that we will use to query the two models.

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
conversation_flant5 = LLMChain(llm=llm_flant5,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the models!

In this example, we are only going to query one claim and see what happens. Of course, feel free to try with different ones.

In [ ]:
filename = 'claims/claim1.json'

# Opening JSON file
claims = {}
with open(filename, 'r') as file:
    data = json.load(file)
claims[filename] = data

# Analyze the claim
print(f"***************************")
print(f"* Claim: {filename}")
print(f"***************************")
print("Original content:")
print("-----------------")
print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
print('Analysis with Azure OpenAI GPT Model:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")
print('Analysis with Flan-T5-Large:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation_flant5.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation_flant5.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation_flant5.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")

As you can see, Flan-T5-Large is relatively fast thanks to it's small size of 770 Million parameters. However the results are less accurate and lacks the detail provided by GPT-4. So it works to some extent, but the results are nowhere near the ones from GPT-4, it's also a bit slower as GPT-4 has a very powerful backend powering it's API.

The art of working with LLM is to find the right balance between the performance and accuracy you require, and the resources it takes along with the involved costs.

Therefore it's important to have confidence checks in place to make sure that as your data changes, or your model evolves, you always get the behaviour you expected.